In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy import io
import wfdb
from scipy import interpolate
from bridge_data_utils import *

import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
def load_normal_ECG(n=256, SNRdB=10, N_test=10_000, val_frac=0.1):
    folder = '/srv/penny/datasets/synth_ecg'
    name = 'ecgSyn_n512_Ny256_(HR 80 100).mat'
    path = os.path.join(folder, name)
    X =  io.loadmat(path)['sigs'].T
    # standardize data
    X = (X - np.mean(X))/X.std()
    # add noise guaranteeing a certain SNR
    if SNRdB is not None:
        X = X + np.random.randn(*X.shape)*10**(-SNRdB/20)
    
    X_train_ = X[:-N_test, 0:n]
    X_test = X[-N_test:, 0:n]
        
    # shuffle the training set
    # and split between training and validation
    indexes = np.random.permutation(len(X_train))
    N_val = int(len(X_train_)*val_frac)
    X_train = X_train_[indexes[:-N_val]]
    X_val = X_train_[indexes[-N_val:]]
    
    return X_train, X_val, X_test
    

In [3]:
def load_anomalous_ECG(n=256, fs_train=256, patient ='101'):
    if 'ecg-anomalies' not in os.listdir():
        os.mkdir('ecg-anomalies')
    # download MIT database and save in 'ecg-anomalies'
    wfdb.dl_database('mitdb', 'ecg-anomalies', records=[patient])
    # retrieve patient's record
    record = wfdb.rdrecord(f'ecg-anomalies/{patient}')
    df = record.to_dataframe()
    
    # label samples in the time series
    annotations = wfdb.rdann(patient, 'atr', pn_dir='mitdb')
    labels_ko = annotations.symbol
    samples = annotations.sample
    labels = np.array(len(df)*['K'])
    labels[samples] = labels_ko
    labels = pd.Series(index=df.index, data=labels)
    
    # resample the time series
    fs = record.fs
    N = fs_train / fs # sampling factor
    time_series = df['MLII'].values
    t = np.linspace(0, 1, len(time_series))
    spline = interpolate.splrep(t, time_series, s=0, k=5)
    tt = np.linspace(0, 1, int(N * len(time_series)))
    time_series_unders = interpolate.splev(tt, spline, der=0)
    time_series_unders = time_series_unders[:(len(time_series_unders)//(2*n))*2*n]
    
    # create a new time index
    tw = pd.Timedelta(1/fs_train, 's')
    index = pd.TimedeltaIndex([df.index[0] + i*tw for i in np.arange(len(time_series_unders))])
    
    # standardize 
    time_series_unders_std = (time_series_unders - time_series_unders.mean()) / time_series_unders.std()
    time_series_unders_std = time_series_unders_std[:(len(time_series_unders_std)//(2*n))*2*n]

    time_series_unders = pd.DataFrame(index=index, data=time_series_unders)
    
    # window the time series
    instances = time_series_unders_std.reshape(-1, n)
    index = index.floor(tw)[::n]
    index = index[:len(instances)] + tw/2    
    instances = pd.DataFrame(index=index, data=instances)
    
    return time_series_unders, instances, labels
    
            

In [4]:
def load_bridge_data(sensor='1D30', n=100, train_period=None, test_period=None):
    # setup data connection
    lib_path = os.path.join('/srv/penny/RFI/roccaprebalza/scripts/old')
    data_path = os.path.join('/srv/penny/RFI/roccaprebalza/rawdata')
    sys.path.append(lib_path)
    from localdata import DataConnection
    rpbz = DataConnection(os.path.join(lib_path, 'index_fixed2.parquet'), data_path)
    
    # sampling frequency
    fs = rpbz.info['fs']
    
    #scaling mode
    mode = 'channel' #channel # all
    # axes
    channels = ['x', 'y', 'z']
    
    channel = '_'.join([ch for ch in channels])
    
    # load train data
    start_time = pd.Timestamp(train_period[0], tz='UTC')
    end_time = pd.Timestamp(train_period[1], tz='UTC')
    time_series_train = rpbz.getData([sensor], start_time, end_time, drop_sync=True).astype(float)
    
    # load test data
    start_time = pd.Timestamp(test_period[0], tz='UTC')
    end_time = pd.Timestamp(test_period[1], tz='UTC')
    time_series_test = rpbz.getData([sensor], start_time, end_time, drop_sync=True).astype(float)
    
    # process data
    threshold = None
    train_data_processed, threshold, scaler = preprocess(time_series_train, n, fs, threshold, None, mode)
    print(threshold)
    # process test data accoding to train data
    test_data_processed, _, _, = preprocess(time_series_test, n, fs, threshold, scaler, mode)
    
    return train_data_processed, test_data_processed, time_series_train, time_series_test

In [7]:
def load_normal_bridge(sensor='1D30', n=100, channel='x', N_test=10_000, val_frac=0.1):
    # definition of the time periods on which train and evaluate PCA
    train_period = ['2017-10-02 00:00:00', '2017-10-08 23:59:59']
    test_period = ['2017-10-09 00:00:00', '2017-10-15 23:59:59']
    train_data, test_data, _, _ = load_bridge_data(sensor, n, train_period, test_period)
    
    N_train = len(train_data)
    X_train_ = train_data[channel].values
    indexes = np.random.permutation(N_train)
    N_val = int(N_train*val_frac)
    X_train = X_train_[indexes[:N_val]]
    X_val = X_train_[indexes[N_val:]]
    
    X_test = test_data[channel].values
    indexes = np.random.permutation(len(X_test))
    X_test = X_test[indexes]
    X_test = X_test[:N_test]
    
    return X_train, X_val, X_test
    


In [ ]:
## TODO

def load_anomalous_bridge(sensor='1D30', n=100, channel='x', N_test=10_000, val_frac=0.1):
    # definition of the time periods on which train and evaluate PCA
    train_period = ['2017-10-02 00:00:00', '2017-10-08 23:59:59']
    test_period = ['2017-11-19 00:00:00', '2017-11-19 23:59:59']
    train_data, test_data, _, _ = load_bridge_data(sensor, n, train_period, test_period)
    
    N_train = len(train_data)
    X_train_ = train_data[channel].values
    indexes = np.random.permutation(N_train)
    N_val = int(N_train*val_frac)
    X_train = X_train_[indexes[:N_val]]
    X_val = X_train_[indexes[N_val:]]
    
    X_test = test_data[channel].values
    indexes = np.random.permutation(len(X_test))
    X_test = X_test[indexes]
    X_test = X_test[:N_test]
    
    return X_train, X_val, X_test
    


In [8]:
X_train, X_val, X_test = load_normal_bridge()

loading /srv/penny/RFI/roccaprebalza/scripts/old/index_fixed2.parquet ... OK
18526.91380556385


In [ ]:
anomalous_period = ['2017-11-19 00:00:00', '2017-11-19 23:59:59']